In [1]:
import scipy as sp 
from matplotlib import pyplot as plt
import cupy as cp
from cupy.linalg import eigh

from tqdm import *
import time
import pennylane as qml
import pennylane.numpy as np 

In [2]:
def generate_Heisenberg_hamiltonian(n):
    """
    Generating Hamiltonian of n-qubit Heisenberg model.

    Args:
        - n (int): The number of qubits 

    Returns:
        - cost_h (qml.Hamiltonian): the hamiltonian of n-qubit Heisenberg model
    """
    coeffs = [1]*3*(n-1)
    obs = []
    for i in range(n-1):
        obs.append(qml.PauliX(i) @ qml.PauliX(i+1) )
        obs.append(qml.PauliY(i) @ qml.PauliY(i+1) )
        obs.append(qml.PauliZ(i) @ qml.PauliZ(i+1) )
    cost_h = qml.Hamiltonian(coeffs, obs)
    return cost_h

In [3]:
def ground_energy(n):
    """ 
    Computing the ground state energy (minimum eigenvalue) of the n-qubit Heisenberg model using CuPy.
    
    Args:
        - n (int): The number of qubits. 

    Returns:
        - min_energy (floats): Minimum eigenvalue of the Heisenberg model.
    
    """
    hamiltonian = generate_Heisenberg_hamiltonian(n)
    hamiltonian_matrix = qml.matrix(hamiltonian)
    h_matrix_cupy = cp.array(hamiltonian_matrix)
    eigv,eigc = eigh(h_matrix_cupy)

    return eigv[0]
    

In [4]:
def qnn_model(params,depth,cost_h):
    def U3_layer(params):
        for i in range(n_wires):
            qml.U3(params[3*i],params[3*i+1],params[3*i+2],wires=i)
    
    def IsingXYZ_layer(params):
        for i in range(n_wires-1):
            qml.IsingXY(params[3*i],wires=[i,(i+1)%n_wires])
            qml.IsingXY(params[3*i+1],wires=[i,(i+1)%n_wires])
            qml.IsingZZ(params[3*i+2],wires=[i,(i+1)%n_wires])
            pass

    def U3_IsingXYZ(params):
        U3_layer(params[0])
        IsingXYZ_layer(params[1])

    qml.layer(U3_IsingXYZ,depth,params[:-1])
    
    return qml.expval(cost_h)

In [28]:
n = 16
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 200
params = init_params
start_time = time.time()
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        end_time = time.time()
        cost_tmp = cost_fn(params)
        duration = (end_time - start_time)
        print("Iter: ", i+1, " | ", "energy value is ", cost_tmp, "  the duration is ", duration, "s")
        if(duration > 30):
            break;

Iter:  1  |  energy value is  -5.9722712164292835   the duration is  1.4094064235687256 s
Iter:  2  |  energy value is  -9.59879015220182   the duration is  3.4110159873962402 s
Iter:  3  |  energy value is  -13.615104771825724   the duration is  5.271894216537476 s
Iter:  4  |  energy value is  -16.577523699767585   the duration is  7.234115362167358 s
Iter:  5  |  energy value is  -18.670306688808694   the duration is  9.226383209228516 s
Iter:  6  |  energy value is  -18.797264825549714   the duration is  11.26986050605774 s
Iter:  7  |  energy value is  -20.016828295054182   the duration is  13.185389280319214 s
Iter:  8  |  energy value is  -20.44764648012978   the duration is  15.239906311035156 s
Iter:  9  |  energy value is  -20.923405602457894   the duration is  17.210930109024048 s
Iter:  10  |  energy value is  -22.27884986087045   the duration is  19.077866554260254 s
Iter:  11  |  energy value is  -22.897960642316498   the duration is  20.962862730026245 s
Iter:  12  |  en

In [29]:
n = 17
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 200
params = init_params
start_time = time.time()
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        end_time = time.time()
        cost_tmp = cost_fn(params)
        duration = (end_time - start_time)
        print("Iter: ", i+1, " | ", "energy value is ", cost_tmp, "  the duration is ", duration, "s")
        if(duration > 85):
            break;

Iter:  1  |  energy value is  -2.9404505600475392   the duration is  2.7079625129699707 s
Iter:  2  |  energy value is  -10.387594685339181   the duration is  6.110752582550049 s
Iter:  3  |  energy value is  -12.83595053810762   the duration is  9.602944612503052 s
Iter:  4  |  energy value is  -15.795066213194236   the duration is  13.355042695999146 s
Iter:  5  |  energy value is  -18.59478865751905   the duration is  17.01283359527588 s
Iter:  6  |  energy value is  -19.00685222010917   the duration is  20.397630214691162 s
Iter:  7  |  energy value is  -20.921501964787147   the duration is  23.975157260894775 s
Iter:  8  |  energy value is  -22.418781028794456   the duration is  27.341576099395752 s
Iter:  9  |  energy value is  -22.582864372280152   the duration is  30.85352325439453 s
Iter:  10  |  energy value is  -23.438491412056518   the duration is  34.28369617462158 s
Iter:  11  |  energy value is  -24.697636181440004   the duration is  37.87183690071106 s
Iter:  12  |  ene

In [5]:
n = 18
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 200
params = init_params
start_time = time.time()
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        end_time = time.time()
        cost_tmp = cost_fn(params)
        duration = (end_time - start_time)
        print("Iter: ", i+1, " | ", "energy value is ", cost_tmp, "  the duration is ", duration, "s")
        if(duration > 329):
            break;

/opt/conda/lib/python3.10/site-packages/autograd/numpy/numpy_vjps.py:698: ComplexWarning: Casting complex values to real discards the imaginary part
  onp.add.at(A, idx, x)


Iter:  1  |  energy value is  -3.9955240279144704   the duration is  5.262375593185425 s
Iter:  2  |  energy value is  -11.791377033456339   the duration is  11.89578104019165 s
Iter:  3  |  energy value is  -13.80765508484244   the duration is  18.594464778900146 s
Iter:  4  |  energy value is  -17.905355011649608   the duration is  25.315962076187134 s
Iter:  5  |  energy value is  -20.017708384479686   the duration is  32.27422022819519 s
Iter:  6  |  energy value is  -20.566225512988098   the duration is  38.99856948852539 s
Iter:  7  |  energy value is  -21.803515906326755   the duration is  45.696842670440674 s
Iter:  8  |  energy value is  -23.29406988843324   the duration is  52.38852620124817 s
Iter:  9  |  energy value is  -23.94062984300338   the duration is  58.98687767982483 s
Iter:  10  |  energy value is  -25.16315859676941   the duration is  65.70987033843994 s
Iter:  11  |  energy value is  -25.748077051573294   the duration is  72.37916254997253 s
Iter:  12  |  energy

In [5]:
n = 19
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 200
params = init_params
start_time = time.time()
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        end_time = time.time()
        cost_tmp = cost_fn(params)
        duration = (end_time - start_time)
        print("Iter: ", i+1, " | ", "energy value is ", cost_tmp, "  the duration is ", duration, "s")
        if(duration > 954):
            break;

/opt/conda/lib/python3.10/site-packages/autograd/numpy/numpy_vjps.py:698: ComplexWarning: Casting complex values to real discards the imaginary part
  onp.add.at(A, idx, x)


Iter:  1  |  energy value is  -3.8881868099712493   the duration is  11.004028797149658 s
Iter:  2  |  energy value is  -11.188249806632916   the duration is  24.811256647109985 s
Iter:  3  |  energy value is  -14.89549677834122   the duration is  38.98478555679321 s
Iter:  4  |  energy value is  -17.390726838929062   the duration is  53.06993222236633 s
Iter:  5  |  energy value is  -19.329735391268862   the duration is  66.9375810623169 s
Iter:  6  |  energy value is  -21.89436158267239   the duration is  80.66530656814575 s
Iter:  7  |  energy value is  -22.69935650621698   the duration is  95.17704272270203 s
Iter:  8  |  energy value is  -23.62799047728496   the duration is  109.25209069252014 s
Iter:  9  |  energy value is  -25.172509163917283   the duration is  123.08505463600159 s
Iter:  10  |  energy value is  -25.854392937802878   the duration is  137.26361894607544 s
Iter:  11  |  energy value is  -26.251843511110575   the duration is  151.0401713848114 s
Iter:  12  |  energ